# Algoritmos de optimización - Seminario<br>
<b>Nombre y Apellidos: </b> Yeray Quiles Ferrández <br>
> 1. Organizar sesiones de doblaje <br>


## Enunciado
<b>Descripción del problema: </b><br>
Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:
- <b>Número de actores: </b> `10`
- <b> Número de tomas: </b> `30`
- <b>Actores/Tomas:</b> https://docs.google.com/spreadsheets/d/1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0/edit?gid=0#gid=0
         

En la tabla: 
- `1` indica que el actor participa en la toma
- `0` en caso contrario          


| Toma | Actor 1 | Actor 2 | Actor 3 | Actor 4 | Actor 5 | Actor 6 | Actor 7 | Actor 8 | Actor 9 | Actor 10 | Total |
|------|---------|---------|---------|---------|---------|---------|---------|---------|---------|----------|-------|
| 1 | 1 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 5 |
| 2 | 0 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 3 |
| 3 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 3 |
| 4 | 1 | 1 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 4 |
| 5 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 3 |
| 6 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 4 |
| 7 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 4 |
| 8 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 3 |
| 9 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 3 |
| 10 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 4 |
| 11 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 5 |
| 12 | 1 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 5 |
| 13 | 1 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 3 |
| 14 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 3 |
| 15 | 1 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 3 |
| 16 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 | 2 |
| 17 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
| 18 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 2 |
| 19 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
| 20 | 1 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 4 |
| 21 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 2 |
| 22 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 4 |
| 23 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
| 24 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 2 |
| 25 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 | 4 |
| 26 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 4 |
| 27 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 2 |
| 28 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
| 29 | 1 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 3 |
| 30 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
|------|---------|---------|---------|---------|---------|---------|---------|---------|---------|----------|-------|
| **TOTAL** | **22** | **14** | **13** | **15** | **11** | **8** | **3** | **4** | **2** | **2** | **94** |


## ¿Cuántas posibilidades hay sin tener en cuenta las restricciones?

Se busca coordinar el doblaje de una película con 30 tomas y 10 actores. Para calcular el número de posibilidades sin restricciones, es necesario estudiar el orden de las tomas a grabar y la distribución de las sesiones de grabación a lo largo de los días.

---

### Opción 1: n! (Permutaciones)

Si únicamente consideramos el **orden de las tomas**:
- Tendríamos n! = 30! permutaciones posibles.
- Este enfoque asume que el orden de grabación es lo único relevante para la planificación.
- No considera cómo se distribuyen las tomas en múltiples días de grabación.
- Sería apropiado únicamente si todas las tomas se grabaran en un solo día consecutivo.

**Resultado:** 265,252,859,812,191,058,636,308,480,000,000 posibilidades

Esta opción, aunque matemáticamente válida para el cálculo de permutaciones, es insuficiente para modelar el problema real de coordinación de doblaje, donde la distribución temporal en días es un factor esencial de la planificación.

### Opción 2: n! × 2^(n-1) (Secuenciación + Distribución)

Esta es la opción escogida para el problema de doblaje, considerando:

#### Orden de las tomas
Si asumimos que el número total de ordenaciones posibles es relevante, entonces se trata de un problema de **permutaciones sin repetición** y el número de posibilidades es **n!**.

#### Distribución de las sesiones de grabación
- En cada sesión de grabación de un día, se pueden grabar un número distinto de tomas k={1,2,…,n}.
- Las grabaciones son únicas, es decir, no pueden repetirse. Por tanto, una vez grabada una toma deberá ser excluida del conjunto de tomas a grabar en las siguientes sesiones.
- Sea d∈{0,…,n−1} el número de particiones en que se pueden distribuir las sesiones de grabación de las tomas. Por ejemplo:
  - Si se graban todas las tomas en un solo día: **d=0** (sin particiones)
  - Si las tomas se graban en dos sesiones: **d=1** (una partición)
  - Y así sucesivamente
- El número de posibles ordenaciones de la partición d=0 sería C(n-1,0), el de la partición d=1 sería C(n-1,1), y así hasta el último número combinatorio.
- Por tanto, el número total de posibles particiones será la suma de todas las posibles distribuciones: **∑(j=0 hasta n-1) C(n-1,j)**

#### Número total de posibilidades sin restricción

El número total de posibles ordenaciones de las tomas a lo largo de los días se obtiene calculando el **producto** del orden de las tomas por la distribución de las sesiones de grabación, que por **identidad binomial** es:

**Núm. posibilidades = n! × ∑(j=0 hasta n-1) C(n-1,j) = n! × 2^(n-1)**

### Comparación de Enfoques

| Aspecto | n! (Opción 1) | n! × 2^(n-1) (Opción 2) |
|---------|---------------|-------------------------|
| **Considera orden de tomas** | ✅ Sí | ✅ Sí |
| **Considera distribución en días** | ❌ No | ✅ Sí |
| **Realismo para doblaje** | ❌ Limitado | ✅ Completo |
| **Resultado para n=30** | ~2.65 × 10^32 | ~1.42 × 10^41 |

### Resultados

| Opción | Fórmula | Resultado |
|--------|---------|-----------|
| **Opción 1 (descartada)** | n! | 265,252,859,812,191,058,636,308,480,000,000 |
| **Opción 2 (correcta)** | n! × 2^(n-1) | 142,406,544,757,979,162,368,320,409,970,933,760,000,000 |

La opción correcta considera tanto el orden de grabación como la distribución temporal, siendo 536,870,912 veces mayor que la opción que solo considera el orden.

**Respuesta final:** 142,406,544,757,979,162,368,320,409,970,933,760,000,000 posibilidades


In [ ]:
import math

# Parámetros del problema
n = 30

# Opción 1: Solo n! (descartada)
solo_factorial = math.factorial(n)

# Opción 2: n! × 2^(n-1) (correcta)
orden_tomas = math.factorial(n)
distribucion_dias = 2**(n-1)
total_posibilidades = orden_tomas * distribucion_dias

# Resultados
print(f'Opción 1 (descartada): n! = {solo_factorial:,}')
print(f'Opción 2 (correcta): n! × 2^(n-1) = {total_posibilidades:,}')
print(f'Factor de diferencia: {total_posibilidades/solo_factorial:,}')

Opción 1 (descartada): n! = 265,252,859,812,191,058,636,308,480,000,000
Opción 2 (correcta): n! × 2^(n-1) = 142,406,544,757,979,162,368,320,409,970,933,760,000,000
Factor de diferencia: 536,870,912.0


## ¿Cuántas posibilidades hay teniendo en cuenta todas las restricciones?

Se busca coordinar el doblaje de una película con 30 tomas y 10 actores, con un máximo de 6 tomas por día y el objetivo de minimizar el costo total. Para calcular el número de posibilidades con restricciones, es necesario estudiar el orden de las tomas a grabar y la distribución de las sesiones de grabación a lo largo de los días.

---

### Opción 1: Combinaciones con orden de días relevante

Si consideramos que el orden cronológico de los días es importante:

#### Orden de las tomas
- Dado que no tenemos restricciones sobre cómo deben ser grabadas las tomas dentro de la sesión de un día, se asume que el orden de grabación de las k tomas dentro de cada día es indiferente.
- Sin embargo, el orden de los días sí se considera relevante (Día 1 ≠ Día 2 ≠ Día 3...).

#### Distribución de las sesiones de grabación
- El mínimo número de sesiones necesarias es **r = ⌈n/k⌉ = ⌈30/6⌉ = 5** días.
- Para optimizar costos, asumimos sesiones de **k = 6** tomas cada una.
- Las grabaciones son únicas: una vez grabada una toma debe ser excluida del conjunto de tomas disponibles.
- Para el día **j+1**, tenemos **C(n-k·j, k)** posibilidades, donde **j ∈ {0,1,2,3,4}**.

#### Número total de posibilidades (orden relevante)

**Núm. posibilidades = ∏[j=0 hasta 4] C(30-6·j, 6)**

**Resultado:** 1,370,874,167,589,326,400 posibilidades

Esta opción distingue entre configuraciones idénticas en términos de costo pero diferentes en orden temporal.

### Opción 2: Combinaciones con orden de días irrelevante

Esta es la opción escogida para el problema de doblaje, considerando:

#### Orden de las tomas
- El orden de grabación de las k tomas dentro de cada día es indiferente.
- El orden de los días tampoco es relevante para el objetivo de minimización de costos.
- Grabar el conjunto {1,2,3,4,5,6} el lunes vs. el martes no afecta el costo total.

#### Distribución de las sesiones de grabación
- Mantenemos la configuración de 5 días con 6 tomas cada uno.
- Las configuraciones que solo difieren en el orden de los días se consideran equivalentes.
- Por ejemplo: [Día A: {1,2,3,4,5,6}, Día B: {7,8,9,10,11,12}] ≡ [Día A: {7,8,9,10,11,12}, Día B: {1,2,3,4,5,6}]

#### Corrección por equivalencia de días

Dado que el orden de los días no importa, debemos dividir el resultado de la Opción 1 por el número de permutaciones de los 5 días:

**Núm. posibilidades = [∏[j=0 hasta 4] C(30-6·j, 6)] ÷ 5!**

### Comparación de Enfoques

| Aspecto | Opción 1 (Orden relevante) | Opción 2 (Orden irrelevante) |
|---------|----------------------------|-------------------------------|
| **Considera restricción k≤6** | ✅ Sí | ✅ Sí |
| **Considera objetivo de costos** | ❌ Parcialmente | ✅ Completamente |
| **Distingue días equivalentes** | ❌ Sí  | ✅ No  |
| **Realismo para doblaje** | ❌ Limitado | ✅ Completo |
| **Resultado para n=30, k=6** | ~1.37 × 10¹⁵ | ~1.14 × 10¹³ |


### Resultados

| Opción | Fórmula | Resultado |
|--------|---------|-----------|
| **Opción 1** | ∏[j=0 hasta 4] C(30-6·j, 6) | 1,370,874,167,589,326,400 |
| **Opción 2** | [∏[j=0 hasta 4] C(30-6·j, 6)] ÷ 5! | 11,423,951,396,577,720 |

La opción correcta evita el sobreconteo de configuraciones equivalentes, siendo 120 veces menor que la opción que distingue incorrectamente el orden de días.


**Respuesta final:** 11,423,951,396,577,720 posibilidades

In [1]:
import math

n = 30
k = 6
num_dias = n // k  # = 5 días

# Cálculo del producto de combinaciones
num_posibilidades_con_orden = 1
for j in range(0, num_dias):
    combinacion = math.comb(n - k*j, k)
    num_posibilidades_con_orden *= combinacion
    print(f"C({n - k*j}, {k}) = {combinacion:,}")

# Corrección por orden irrelevante de días
factorial_dias = math.factorial(num_dias)
num_posibilidades_sin_orden = num_posibilidades_con_orden // factorial_dias

print(f"Con orden: {num_posibilidades_con_orden:,}")
print(f"Factorial 5!: {factorial_dias}")
print(f"Sin orden: {num_posibilidades_sin_orden:,}")


C(30, 6) = 593,775
C(24, 6) = 134,596
C(18, 6) = 18,564
C(12, 6) = 924
C(6, 6) = 1
Con orden: 1,370,874,167,589,326,400
Factorial 5!: 120
Sin orden: 11,423,951,396,577,720


## ¿Cual es la estructura de datos que mejor se adapta al problema?

### Primera aproximación para definir la estructura de los datos:

**Datos de entrada**

Inicialmente se pensó en trabajar con la matriz de datos como un **array bidimensional de numpy** o un **dataframe de pandas** de manera que:
* en las filas tuviese como índice el número de toma (1 a 30),
* en las columnas tuviese como nombre el número de actor (1 a 10) y  
* en cada valor de la matriz el indicador ∈ {1 si en la toma i-ésima participa el actor j-ésimo, 0 en caso contrario}


**Datos de salida**

La solución se presentaría como una lista de sublistas donde cada sublista estaría formada por un número de tomas (≤6 tomas) para una sesión específica y se presentaría tantas sublistas como sesiones de grabación se considerase. Por ejemplo:

`[[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24], [25, 26, 27, 28, 29, 30]]`

No obstante, al experimentar las primeras dificultades en los diferentes bucles de optimización y analizar el rendimiento de la primera aproximación propuesta, se tomó la decisión de considerar una estructura alternativa.

### Problemas identificados con la aproximación inicial:

Durante la implementación inicial surgieron varios problemas:

1. **Ineficiencia en consultas frecuentes**: La operación más común "¿qué actores necesito para las tomas {1,5,12,18,23,29}?" requería iterar por cada toma y cada actor (6×10=60 operaciones), cuando conceptualmente solo necesitaba unir 6 conjuntos pequeños.

2. **Desperdicio masivo de memoria**: Del total de 300 posiciones en la matriz (30×10), solo 94 contenían el valor 1, significando que el 68.7% del espacio almacenaba ceros innecesarios.

3. **Complejidad en operaciones de conjuntos**: Calcular la unión de actores para múltiples tomas requería bucles anidados y creación manual de conjuntos intermedios, complicando el código y ralentizando las evaluaciones.

4. **Escalabilidad limitada**: Para problemas mayores, el desperdicio de memoria crecería proporcionalmente, haciendo inviable la estructura para instancias más grandes.

### Estructura final de los datos:

**Datos de entrada**

En base a la matriz descrita en el apartado anterior se llevó a cabo una transformación para montar una **estructura híbrida de diccionarios con conjuntos** donde:
* cada clave es un número de toma (1 a 30) y
* cada valor del diccionario es un **conjunto (set)** con los números de actores que participan en esa toma.

Por ejemplo:
```python
{
    1: {1, 2, 3, 4, 5},     # Toma 1: actores 1,2,3,4,5
    2: {3, 4, 5},           # Toma 2: actores 3,4,5  
    3: {2, 5, 7},           # Toma 3: actores 2,5,7
    # ... hasta toma 30
}
```

Esta estructura elimina completamente el almacenamiento de ceros y aprovecha las operaciones nativas de conjuntos de Python para uniones eficientes.

**Datos de salida**

Se mantiene el formato de lista de sublistas para las configuraciones, pero ahora el cálculo del costo asociado es significativamente más eficiente:

```python
def calcular_costo_configuracion(configuracion):
    costo_total = 0
    for dia in configuracion:
        actores_dia = set()
        for toma in dia:
            actores_dia.update(tomas_actores[toma])  # Unión eficiente
        costo_total += len(actores_dia)
    return costo_total
```

### Ventajas de la estructura final:

1. **Memoria optimizada**: Solo almacena las 94 participaciones reales vs 300 posiciones de la matriz.

2. **Operaciones más rápidas**: La consulta crítica se reduce de O(k×m) a O(k) operaciones.

3. **Código más limpio**: Las operaciones de conjuntos son naturales en Python y eliminan bucles complejos.

4. **Escalabilidad natural**: El crecimiento es proporcional a las participaciones reales, no al producto cartesiano teórico.

### Limitaciones

Debido a la magnitud de los cálculos requeridos para evaluar **11,423,951,396,577,720 configuraciones posibles** y las restricciones computacionales, se introdujeron varias consideraciones:

1. **Se asumió que el orden de los días no es relevante** para el objetivo de minimización de costos, dividiendo el espacio de soluciones por 5! para evitar sobreconteo de configuraciones equivalentes.

2. **Se estableció uniformidad en el número de tomas por sesión** (k=6) para la configuración óptima, aunque la estructura permite flexibilidad para otras distribuciones.

3. **Se priorizó simplicidad sobre optimización extrema**, manteniendo el código legible y mantenible en lugar de implementar estructuras más complejas pero marginalmente más rápidas.


La estructura final representa un equilibrio óptimo entre eficiencia computacional, simplicidad implementativa y escalabilidad para el problema específico de coordinación de doblaje planteado.

## ¿Cual es la función objetivo?


## ¿Es un problema de maximización o minimización?

## Diseña un algoritmo para resolver el problema por fuerza bruta

## Calcula la complejidad del algoritmo por fuerza bruta

## Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

## Calcula la complejidad del algoritmo

## Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

## Aplica el algoritmo al juego de datos generado

## Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

## Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño